# Lab Assignment 8: Data Management Using `pandas`, Part 1
## DS 6001: Practice and Application of Data Science

### Instructions
Please answer the following questions as completely as possible using text, code, and the results of code as needed. Format your answers in a Jupyter notebook. To receive full credit, make sure you address every part of the problem, and make sure your document is formatted in a clean and professional way.

In this lab, you will be working with the [2017 Workplace Health in America survey](https://www.cdc.gov/workplacehealthpromotion/survey/data.html) which was conducted by the Centers for Disease Control and Prevention. According to the survey's [guidence document](https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/2017-WHA-Guidance-Document-for-Use-of-Public-Data-files-508.pdf):

> The Workplace Health in America (WHA) Survey gathered information from a cross-sectional, nationally representative sample of US worksites. The sample was drawn from the Dun & Bradstreet (D&B) database of all private and public employers in the United States with at least 10 employees. Like previous national surveys, the worksite served as the sampling unit rather than the companies or firms to which the worksites belonged. Worksites were selected using a stratified simple random sample (SRS) design, where the primary strata were ten multi-state regions defined by the Centers for Disease Control and Prevention (CDC), plus an additional stratum containing all hospital worksites. 

The data contain over 300 features that report the industry and type of company where the respondents are employed, what kind of health insurance and other health programs are offered, and other characteristics of the workplaces including whether employees are allowed to work from home and the gender and age makeup of the workforce. The data are full of interesting information, but in order to make use of the data a great deal of data manipulation is required first.

## Problem 0
Import the following libraries:

In [29]:
import numpy as np
import pandas as pd
import sidetable
import sqlite3
import warnings
import os
warnings.filterwarnings('ignore')

## Problem 1
The raw data are stored in an ASCII file on the 2017 Workplace Health in America survey [homepage](https://www.cdc.gov/workplacehealthpromotion/survey/data.html). Load the raw data directly into Python without downloading the data onto your harddrive and display a dataframe with only the 14th, 28th, and 102nd rows of the data. [1 point]

In [30]:
url = "https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/whpps_120717.csv"
data = pd.read_csv(url)

url2 = "https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/WHA_120717.sas7bdat"
data2 = pd.read_sas(url2)

data2.head()

# display the 14th, 28th, and 102nd rows

selected_rows = data2.iloc[[13, 27, 101]]

selected_rows

,OC1,OC3,HI1,HI2,HI3,HI4,HRA1,HRA1A,HRA1B,HRA1E,...,WL3_05,E1_09,Suppquex,Id,Region,CDC_Region,Industry,Size,Varstrata,Finalwt_worksite
13,3.0,1.0,2.0,3.0,2.0,1.0,1.0,3.0,3.0,1.0,...,NaN,NaN,1.0,b'0000001437',4.0,6.0,7.0,3.0,0.0,47.793941
27,1.0,3.0,1.0,3.0,1.0,1.0,1.0,2.0,4.0,2.0,...,NaN,NaN,1.0,b'0000002501',2.0,4.0,7.0,8.0,0.0,47.793941
101,2.0,1.0,1.0,3.0,2.0,1.0,1.0,2.0,4.0,2.0,...,NaN,NaN,2.0,b'0000012636',4.0,6.0,7.0,4.0,0.0,47.793941


## Problem 2 
The data contain 301 columns. Create a new variable in Python's memory to store a working version of the data. In the working version, delete all of the columns except for the following:

* `Industry`: 7 Industry Categories with NAICS codes

* `Size`: 8 Employee Size Categories

* `OC3` Is your organization for profit, non-profit, government?

* `HI1` In general, do you offer full, partial or no payment of premiums for personal health insurance for full-time employees?

* `HI2` Over the past 12 months, were full-time employees asked to pay a larger proportion, smaller proportion or the same proportion of personal health insurance premiums?

* `HI3`: Does your organization offer personal health insurance for your part-time employees?

* `CP1`: Are there health education programs, which focus on skill development and lifestyle behavior change along with information dissemination and awareness building?

* `WL6`: Allow employees to work from home?

* Every column that begins `WD`, expressing the percentage of employees that have certain characteristics at the firm

[1 point]

In [31]:
# Create a working version of the data
working_data = data2.copy()

# keep specified columns
columns_to_keep = [
    'Industry',
    'Size',
    'OC3',
    'HI1',
    'HI2',
    'HI3',
    'CP1',
    'WL6'
]

# add columns starting with "WD"
wd_columns = [col for col in working_data.columns if col.startswith("WD")]
columns_to_keep.extend(wd_columns)

# update for specified columns
working_data = working_data[columns_to_keep]

In [32]:
print(working_data.columns)

Index(['Industry', 'Size', 'OC3', 'HI1', 'HI2', 'HI3', 'CP1', 'WL6', 'WD1_1',
       'WD1_2', 'WD2', 'WD3', 'WD4', 'WD5', 'WD6', 'WD7'],
      dtype='object')


## Problem 3
The [codebook](https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/2017-WHA-Datafile-Codebook-508.pdf) for the WHA data contain short descriptions of the meaning of each of the columns in the data. Use these descriptions to decide on better and more intuitive names for the columns in the working version of the data, and rename the columns accordingly. [1 point]

In [33]:
# dictionary to map old column names to new column names
column_name_mapping = {
    'Industry': 'Industry_Category',
    'Size': 'Employee_Size_Category',
    'OC3': 'Organization_Type',
    'HI1': 'Health_Insurance_Premium_Payment',
    'HI2': 'Premium_Proportion_Change',
    'HI3': 'Insurance_Part_Time_Employees',
    'CP1': 'Health_Education_Programs',
    'WL6': 'Work_From_Home_Allowed',
    'WD1_1': 'Pct_Employees_U30',
    'WD1_2': 'Pct_Employees_O60',
    'WD2': 'Pct_Employees_Female',
    'WD3': 'Pct_Employees_Hourly',
    'WD4': 'Pct_Employees_NonDaytime_Reg',
    'WD5': 'Pct_Employees_Remote',
    'WD6': 'Pct_Employees_Union',
    'WD7': 'Pct_Avg_Annual_Turnover'
}

# rename columns
working_data.rename(columns=column_name_mapping, inplace=True)

In [34]:
print(working_data.columns)

Index(['Industry_Category', 'Employee_Size_Category', 'Organization_Type',
       'Health_Insurance_Premium_Payment', 'Premium_Proportion_Change',
       'Insurance_Part_Time_Employees', 'Health_Education_Programs',
       'Work_From_Home_Allowed', 'Pct_Employees_U30', 'Pct_Employees_O60',
       'Pct_Employees_Female', 'Pct_Employees_Hourly',
       'Pct_Employees_NonDaytime_Reg', 'Pct_Employees_Remote',
       'Pct_Employees_Union', 'Pct_Avg_Annual_Turnover'],
      dtype='object')


## Problem 4
Using the codebook and this [dictionary of NAICS industrial codes](https://www.naics.com/search-naics-codes-by-industry/), place descriptive labels on the categories of the industry column in the working data. [1 point]

In [39]:
print(working_data.Industry_Category.unique())

[7. 1. 2. 3. 4. 5. 6.]


In [40]:
# dictionary to map industry categories descriptions
industry_labels = {
    1: 'Agriculture, Forestry, Fishing and Hunting; Mining, Quarrying, and Oil and Gas Extraction; Utilities; Construction; Manufacturing',
    2: 'Wholesale Trade; Retail Trade; Transportation and Warehousing',
    3: 'Arts, Entertainment, and Recreation; Accommodation and Food Services; Other Services (except Public Administration)',
    4: 'Information; Finance and Insurance; Real Estate and Rental and Leasing; Professional, Scientific, and Technical Services; Management of Companies and Enterprises; Administrative and Support and Waste Management and Remediation Services',
    5: 'Educational Services; Health Care and Social Assistance (excluding hospital worksites)',
    6: 'Public Administration',
    7: 'Hospital Worksites'
}

# replace the industry codes with descriptions
working_data['Industry_Category'] = working_data['Industry_Category'].replace(industry_labels)

In [43]:
working_data['Industry_Category']

0          Hospital Worksites
1          Hospital Worksites
2          Hospital Worksites
3          Hospital Worksites
4          Hospital Worksites
                ...          
2838    Public Administration
2839    Public Administration
2840    Public Administration
2841    Public Administration
2842    Public Administration
Name: Industry_Category, Length: 2843, dtype: object

## Problem 5
Using the codebook, recode the "size" column to have three categories: "Small" for workplaces with fewer than 100 employees, "Medium" for workplaces with at least 100 but fewer than 500 employees, and "Large" for companies with at least 500 employees. [Note: Python dataframes have an attribute `.size` that reports the space the dataframe takes up in memory. Don't confuse this attribute with the column named "Size" in the raw data.] [1 point]

In [45]:
working_data['Employee_Size_Category'].unique()

array([7., 6., 8., 4., 1., 3., 5., 2.])

In [46]:
'''1 = Size Category 1: 10-24 
2 = Size Category 2: 25-49 
3 = Size Category 3: 50-99 
4 = Size Category 4: 100-249 
5 = Size Category 5: 250-499
6 = Size Category 6: 500-749 
7 = Size Category 7: 750-999
8 = Size Category 8: 1,000+ '''

# dictionary to map size categories to small, medium, or large
size_labels = {
    1: 'Small',
    2: 'Small',
    3: 'Small',
    4: 'Medium',
    5: 'Medium',
    6: 'Large',
    7: 'Large',
    8: 'Large'
}

# replace the size codes with descriptions
working_data['Employee_Size_Category'] = working_data['Employee_Size_Category'].replace(size_labels)

In [47]:
working_data['Employee_Size_Category']

0        Large
1        Large
2        Large
3       Medium
4       Medium
         ...  
2838    Medium
2839    Medium
2840     Large
2841     Large
2842     Large
Name: Employee_Size_Category, Length: 2843, dtype: object

## Problem 6
Use the codebook to write accurate and descriptive labels for each category for each categorical column in the working data. Then apply all of these labels to the data at once. Code "Legitimate Skip", "Don't know", "Refused", and "Blank" as missing values. [2 points]

In [48]:
category_labels = {
    'Organization_Type': {
        1: 'For profit, public',
        2: 'For profit, private',
        3: 'Non-profit',
        4: 'State or local government',
        5: 'Federal government',
        6: 'Other',
        97: np.nan,
        98: np.nan,
        99: np.nan
    },
    'Health_Insurance_Premium_Payment': {
        1: 'Full insurance',
        2: 'Partial insurance',
        3: 'No insurance',
        97: np.nan,
        98: np.nan,
        99: np.nan
    },
    'Premium_Proportion_Change': {
        1: 'Larger',
        2: 'Smaller',
        3: 'About the same',
        97: np.nan,
        98: np.nan,
        99: np.nan
    },
    'Insurance_Part_Time_Employees': {
        1: 'Yes',
        2: 'No',
        97: np.nan,
        98: np.nan,
        99: np.nan
    },
    'Health_Education_Programs': {
        1: 'Yes',
        2: 'No',
        97: np.nan,
        98: np.nan,
        99: np.nan
    },
    'Work_From_Home_Allowed': {
        1: 'Yes',
        2: 'No',
        97: np.nan,
        98: np.nan,
        99: np.nan
    }
}

working_data = working_data.replace(category_labels)

## Problem 7
The features that measure the percent of the workforce with a particular characteristic use the codes 997, 998, and 999 to represent "Don't know", "Refusal", and "Blank/Invalid" respectively. Replace these values with missing values for all of the percentage features at the same time. [1 point]

In [50]:
percentage_columns = [
    'Pct_Employees_U30',
    'Pct_Employees_O60',
    'Pct_Employees_Female',
    'Pct_Employees_Hourly',
    'Pct_Employees_NonDaytime_Reg',
    'Pct_Employees_Remote',
    'Pct_Employees_Union',
    'Pct_Avg_Annual_Turnover'
]

missing_value_codes = {997: np.nan, 998: np.nan, 999: np.nan}

working_data[percentage_columns] = working_data[percentage_columns].replace(missing_value_codes)

## Problem 8
Sort the working data by industry in ascending alphabetical order. Within industry categories, sort the rows by size in ascending alphabetical order. Within groups with the same industry and size, sort by percent of the workforce that is under 30 in descending numeric order. [1 point]

In [55]:
# sort data

working_data.sort_values(['Industry_Category', 'Employee_Size_Category', 'Pct_Employees_U30'],
    ascending=[True, True, False]
)

,Industry_Category,Employee_Size_Category,Organization_Type,Health_Insurance_Premium_Payment,Premium_Proportion_Change,Insurance_Part_Time_Employees,Health_Education_Programs,Work_From_Home_Allowed,Pct_Employees_U30,Pct_Employees_O60,Pct_Employees_Female,Pct_Employees_Hourly,Pct_Employees_NonDaytime_Reg,Pct_Employees_Remote,Pct_Employees_Union,Pct_Avg_Annual_Turnover
1732,"Agriculture, Forestry, Fishing and Hunting; Mi...",Large,"For profit, private",Partial insurance,About the same,No,Yes,No,50.0,10.0,50.0,75.0,10.0,0.0,0.0,75.0
1476,"Agriculture, Forestry, Fishing and Hunting; Mi...",Large,"For profit, private",Partial insurance,About the same,No,Yes,No,40.0,10.0,30.0,60.0,30.0,5.0,0.0,10.0
1477,"Agriculture, Forestry, Fishing and Hunting; Mi...",Large,"For profit, private",Partial insurance,Smaller,No,Yes,Yes,25.0,15.0,20.0,60.0,10.0,2.0,60.0,5.0
704,"Agriculture, Forestry, Fishing and Hunting; Mi...",Large,"For profit, private",Full insurance,About the same,No,Yes,Yes,20.0,15.0,17.0,62.0,10.0,5.0,0.0,11.0
1241,"Agriculture, Forestry, Fishing and Hunting; Mi...",Large,"For profit, private",Full insurance,About the same,No,Yes,Yes,20.0,25.0,50.0,70.0,20.0,5.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595,Wholesale Trade; Retail Trade; Transportation ...,Small,"For profit, private",Full insurance,About the same,Yes,No,No,NaN,2.0,2.0,90.0,0.0,0.0,0.0,0.0
2604,Wholesale Trade; Retail Trade; Transportation ...,Small,Non-profit,Full insurance,About the same,No,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2626,Wholesale Trade; Retail Trade; Transportation ...,Small,"For profit, private",Partial insurance,Larger,Yes,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2629,Wholesale Trade; Retail Trade; Transportation ...,Small,"For profit, public",Full insurance,Larger,No,Yes,Yes,NaN,2.0,15.0,NaN,NaN,90.0,0.0,15.0


## Problem 9
There is one row in the working data that has a `NaN` value for industry. Delete this row. Use a logical expression, and not the row number. [1 point]

In [57]:
# drop row with NaN value
working_data.dropna(subset=['Industry_Category'])

working_data['Industry_Category'].isnull().values.any()

False

## Problem 10
Create a new feature named `gender_balance` that has three categories: "Mostly men" for workplaces with between 0% and 35% female employees, "Balanced" for workplaces with more than 35% and at most 65% female employees, and "Mostly women" for workplaces with more than 65% female employees. [1 point]

In [58]:
def create_gender_balance(Pct_Employees_Female):
    if 0 <= Pct_Employees_Female <= 35:
        return 'Mostly men'
    elif 35 < Pct_Employees_Female <= 65:
        return 'Balanced'
    elif Pct_Employees_Female > 65:
        return 'Mostly women'
    else:
        return np.nan

# create gender_balance feature with function
working_data['gender_balance'] = working_data['Pct_Employees_Female'].apply(create_gender_balance)

In [59]:
working_data['gender_balance'].head()

0    Mostly women
1    Mostly women
2             NaN
3        Balanced
4        Balanced
Name: gender_balance, dtype: object

## Problem 11
Change the data type of all categorical features in the working data from "object" to "category". [1 point]

In [63]:
# categorical columns
categorical_columns = [
    'Industry_Category',
    'Employee_Size_Category',
    'Organization_Type',
    'Health_Insurance_Premium_Payment',
    'Premium_Proportion_Change',
    'Insurance_Part_Time_Employees',
    'Health_Education_Programs',
    'Work_From_Home_Allowed',
    'gender_balance'
]

# dictionary specifying data types
column_data_types = {column: 'category' for column in categorical_columns}

# change data type
working_data = working_data.astype(column_data_types)

In [64]:
working_data.dtypes

Industry_Category                   category
Employee_Size_Category              category
Organization_Type                   category
Health_Insurance_Premium_Payment    category
Premium_Proportion_Change           category
Insurance_Part_Time_Employees       category
Health_Education_Programs           category
Work_From_Home_Allowed              category
Pct_Employees_U30                    float64
Pct_Employees_O60                    float64
Pct_Employees_Female                 float64
Pct_Employees_Hourly                 float64
Pct_Employees_NonDaytime_Reg         float64
Pct_Employees_Remote                 float64
Pct_Employees_Union                  float64
Pct_Avg_Annual_Turnover              float64
gender_balance                      category
dtype: object

## Problem 12
Filter the data to only those rows that represent small workplaces that allow employees to work from home. Then report how many of these workplaces offer full insurance, partial insurance, and no insurance. Use a function that reports the percent, cumulative count, and cumulative percent in addition to the counts. [1 point]

In [67]:
small_wfh = working_data.query('Employee_Size_Category == "Small" and Work_From_Home_Allowed == "Yes"').stb.freq(['Health_Insurance_Premium_Payment'])

small_wfh

,Health_Insurance_Premium_Payment,count,percent,cumulative_count,cumulative_percent
0,Full insurance,324,46.285714,324,46.285714
1,Partial insurance,310,44.285714,634,90.571429
2,No insurance,66,9.428571,700,100.000000


## Problem 13
Anything that can be done in SQL can be done with `pandas`. The next several questions ask you to write `pandas` code to match a given SQL query. But to check that the SQL query and `pandas` code yield the same result, create a new database wsing the `sqlite3` package and input the cleaned WHA data as a table in this database. (See module 6 for a discussion of SQlite in Python.) [1 point]

In [68]:
# new SQLite database
conn = sqlite3.connect('wha_data.db')

# input the data as table in db
working_data.to_sql('wha_data', conn, if_exists = 'replace', index = False)

2843

## Problem 14
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT size, type, premiums AS insurance, percent_female FROM whpps
WHERE industry = 'Hospitals' AND premium_change='Smaller'
ORDER BY percent_female DESC;
```
For each of these queries, your feature names might be different from the ones listed in the query, depending on the names you chose in problem 3.
[2 points]

In [71]:
# filter and sort data 
result = working_data.query("Industry_Category == 'Hospital Worksites' and Premium_Proportion_Change == 'Smaller'")\
    .sort_values('Pct_Employees_Female', ascending=False)\
    .loc[:, ['Employee_Size_Category', 'Organization_Type', 'Health_Insurance_Premium_Payment', 'Pct_Employees_Female']]

result

,Employee_Size_Category,Organization_Type,Health_Insurance_Premium_Payment,Pct_Employees_Female
320,Medium,Non-profit,Full insurance,89.0
187,Large,Non-profit,Partial insurance,80.0
214,Large,Non-profit,Partial insurance,80.0
229,Small,Non-profit,Full insurance,75.0
191,Medium,Non-profit,Partial insurance,65.0
3,Medium,"For profit, private",Full insurance,50.0
11,Medium,NaN,Partial insurance,NaN
48,Medium,Non-profit,Partial insurance,NaN
51,Medium,Non-profit,Full insurance,NaN
75,Medium,Non-profit,Full insurance,NaN


## Problem 15
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT industry, 
    AVG(percent_female) as percent_female, 
    AVG(percent_under30) as percent_under30,
    AVG(percent_over60) as percent_over60
FROM whpps
GROUP BY industry
ORDER BY percent_female DESC;
```
[2 points]

In [72]:
# group, aggregate, and sort data
result2 = working_data.groupby('Industry_Category')\
    .agg({
        'Pct_Employees_Female': 'mean',
        'Pct_Employees_U30': 'mean',
        'Pct_Employees_O60': 'mean'
    })\
    .sort_values('Pct_Employees_Female', ascending=False)\
    .reset_index()

result2

,Industry_Category,Pct_Employees_Female,Pct_Employees_U30,Pct_Employees_O60
0,Educational Services; Health Care and Social A...,80.657143,25.745665,11.349570
1,Hospital Worksites,76.427027,27.213793,16.489655
2,"Arts, Entertainment, and Recreation; Accommoda...",53.804416,38.566343,11.544872
3,Information; Finance and Insurance; Real Estat...,50.632184,23.821752,12.465465
4,Public Administration,39.056738,21.015625,15.015385
5,Wholesale Trade; Retail Trade; Transportation ...,32.657258,29.108696,12.584034
6,"Agriculture, Forestry, Fishing and Hunting; Mi...",20.328605,22.257143,10.690355


## Problem 16
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT gender_balance, premiums, COUNT(*)
FROM whpps
GROUP BY gender_balance, premiums
HAVING gender_balance is NOT NULL and premiums is NOT NULL;
```
[2 points]

In [75]:
# group, count, and filter data
result3 = working_data.groupby(['gender_balance', 'Health_Insurance_Premium_Payment'], dropna = True)\
    .size()\
    .reset_index(name = 'count')

result3

,gender_balance,Health_Insurance_Premium_Payment,count
0,Balanced,Full insurance,226
1,Balanced,No insurance,77
2,Balanced,Partial insurance,271
3,Mostly men,Full insurance,301
4,Mostly men,No insurance,91
5,Mostly men,Partial insurance,332
6,Mostly women,Full insurance,267
7,Mostly women,No insurance,107
8,Mostly women,Partial insurance,333
